In [1]:
import pycaret
from pycaret.classification import *
import pandas as pd

In [2]:
df = pd.read_pickle("../data/depresjon/depresjon_preprocessed.pkl")
df

user_id  activity  days  gender  age  afftype  melanch  inpatient edu  \
0            1         0    11       2    1      2.0      2.0        2.0   1   
1            1       143    11       2    1      2.0      2.0        2.0   1   
2            1         0    11       2    1      2.0      2.0        2.0   1   
3            1        20    11       2    1      2.0      2.0        2.0   1   
4            1       166    11       2    1      2.0      2.0        2.0   1   
...        ...       ...   ...     ...  ...      ...      ...        ...  ..   
551711       9         0    13       2    2      1.0      2.0        2.0   1   
551712       9         0    13       2    2      1.0      2.0        2.0   1   
551713       9       449    13       2    2      1.0      2.0        2.0   1   
551714       9         0    13       2    2      1.0      2.0        2.0   1   
551715       9         0    13       2    2      1.0      2.0        2.0   1   

        marriage  ...  month_sin  weekday_sin  week_sin   day_sin  month_cos  \
0            1.0  ...  -0.974928     0.974928 -0.998027  0.988468  -0.222521   
1            1.0  ...  -0.974928     0.974928 -0.998027  0.988468  -0.222521   
2            1.0  ...  -0.974928     0.974928 -0.998027  0.988468  -0.222521   
3            1.0  ...  -0.974928     0.974928 -0.998027  0.988468  -0.222521   
4            1.0  ...  -0.974928     0.974928 -0.998027  0.988468  -0.222521   
...          ...  ...        ...          ...       ...       ...        ...   
551711       1.0  ...  -0.781831     0.433884  -0.24869  0.897805   0.623490   
551712       1.0  ...  -0.781831     0.433884  -0.24869  0.897805   0.623490   
551713       1.0  ...  -0.781831     0.433884  -0.24869  0.897805   0.623490   
551714       1.0  ...  -0.781831     0.433884  -0.24869  0.897805   0.623490   
551715       1.0  ...  -0.781831     0.433884  -0.24869  0.897805   0.623490   

        weekday_cos  week_cos   day_cos  activity_quantile  \
0         -0.222521  0.062791  0.151428                  0   
1         -0.222521  0.062791  0.151428                  1   
2         -0.222521  0.062791  0.151428                  0   
3         -0.222521  0.062791  0.151428                  1   
4         -0.222521  0.062791  0.151428                  1   
...             ...       ...       ...                ...   
551711    -0.900969  0.968583 -0.440394                  1   
551712    -0.900969  0.968583 -0.440394                  1   
551713    -0.900969  0.968583 -0.440394                  2   
551714    -0.900969  0.968583 -0.440394                  1   
551715    -0.900969  0.968583 -0.440394                  1   

        user_activity_quantile  
0                            0  
1                            1  
2                            0  
3                            1  
4                            1  
...                        ...  
551711                       1  
551712                       1  
551713                       2  
551714                       1  
551715                       1  

[529944 rows x 23 columns]

In [3]:
df.columns

Index(['user_id', 'activity', 'days', 'gender', 'age', 'afftype', 'melanch',
       'inpatient', 'edu', 'marriage', 'work', 'is_weekend', 'category_madrs',
       'month_sin', 'weekday_sin', 'week_sin', 'day_sin', 'month_cos',
       'weekday_cos', 'week_cos', 'day_cos', 'activity_quantile',
       'user_activity_quantile'],
      dtype='object')

In [4]:
def train_test_split_per_user(data, train_size=0.7):
    users = list(set(data.user_id))
    users = sorted(users, reverse=True)  # fix randomness
    total_users = len(users)
    slice = int(train_size * total_users)
    users_train = users[:slice]
    users_test = users[slice:]
    return data[data.user_id.isin(users_train)], data[data.user_id.isin(users_test)]

In [5]:
train_data, test_data = train_test_split_per_user(df)

fold_groups = train_data.user_id
train_data = train_data.drop(columns=['user_id'])
test_data = test_data.drop(columns=['user_id'])

In [6]:
s = setup(data=train_data, target='category_madrs', session_id=123, 
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data)

In [7]:
best = compare_models(sort = 'acc') 